# **Multigrams: Full Workflow**
The multigram workflow mirrors the unigram workflow, but with two differences. First, instead of _creating_ a whitelist, you filter the multigram corpus _using_ a whitelist containing the top-_N_ unigrams. Second, the multigram workflow adds a **pivoting** step. Pivoting reorganizes the database so that it's easy to query year-ngram combinations. For instance, you can learn how many times the word "nuclear" appeared in 2011 by querying the key `[2011] nuclear`. This is useful for analyzing changes in word meanings over time.

## **Setup**
### Imports

In [2]:
%load_ext autoreload
%autoreload 2

from ngramkit.ngram_acquire import download_and_ingest_to_rocksdb
from ngramkit.ngram_filter.pipeline.orchestrator import build_processed_db
from ngramkit.ngram_pivot.pipeline import build_pivoted_db
from ngramkit.utilities.peek import db_head, db_peek, db_peek_prefix

### Configure

In [3]:
db_path_stub = '/scratch/edk202/NLP_corpora/Google_Books/'
archive_path_stub = '/scratch/edk202/NLP_archive/Google_Books/'
release = '20200217'
language = 'eng'
size = 5

## **Step 1: Download and Ingest**

In [3]:
download_and_ingest_to_rocksdb(
    ngram_size=size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    archive_path_stub=None,
    ngram_type="tagged",
    random_seed=67,
    overwrite_db=True,
    workers=100,
    write_batch_size=5_000,
    open_type="write:packed24",
    compact_after_ingest=False
)

N-GRAM ACQUISITION PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-17 19:23:42

Download Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Ngram repo:           https://books.storage.googleapis.com/?prefix=ngrams/books/20200217/eng/5-
DB path:              /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
File range:           0 to 19422
Total files:          19423
Files to get:         19423
Skipping:             0
Download workers:     100
Batch size:           5,000
Ngram size:           5
Ngram type:           tagged
Overwrite DB:         True
DB Profile:           write:packed24

Download Progress
════════════════════════════════════════════════════════════════════════════════════════════════════


Files Processed: 100%|█████████████████████████████████████████████████| 19423/19423 [3:03:48<00:00]



Processing complete!

Final Summary
════════════════════════════════════════════════════════════════════════════════════════════════════
Fully processed files:       19423
Failed files:                0
Total entries written:       2,480,130,431
Write batches flushed:       6456
Uncompressed data processed: 28.44 TB
Processing throughput:       2698.81 MB/sec

End Time: 2025-11-17 22:27:50.939221
Total Runtime: 3:04:08.584232
Time per file: 0:00:00.568840
Files per hour: 6328.7


## **Step 2: Filter and Normalize**
`config.py` contains generic defaults for the filtering pipeline. You can override these defaults by passing option dictionaries to the `build_processed_db` function, as seen below. As implemented here, we use the whitelist from the unigram workflow to filter the multigram corpus. If we weren't using a whitelist, we could normalize, filter, and lemmatize each token just as we did for the unigrams.

In [3]:
whitelist_options = {
    'whitelist_path': f'{db_path_stub}/{release}/{language}/1gram_files/1grams_processed.db/whitelist.txt',
    'output_whitelist_path': None
}

build_processed_db(
    mode="restart",
    ngram_size=size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    num_workers=40,
    num_initial_work_units=600,
    cache_partitions=True,
    use_cached_partitions=True,
    progress_every_s=10.0,
    **whitelist_options
);


N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-18 02:39:21
Mode:       RESTART

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
Target DB:            ...dk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Temp directory:       ...tch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/processing_tmp

Parallelism
────────────────────────────────────────────────────────────────────────────────────────────────────
Workers:              40
Initial work units:   600

Database Profiles
────────────────────────────────────────────────────────────────────────────────────────────────────
Reader profile:       read:packed24
Writer profile:       write:packed24

Ingestion Configuration
────────────────────────

Shards Ingested: 100%|███████████████████████████████████████████████████████| 600/600 [43:08<00:00]



Ingestion complete: 600 shards, 454,491,124 items in 2588.4s (175,587 items/s)

Phase 4: Finalizing database...
════════════════════════════════════════════════════════════════════════════════════════════════════

Post-Ingestion Compaction
────────────────────────────────────────────────────────────────────────────────────────────────────
Initial DB size:         443.99 GB
Compaction completed in 0:10:17
Size before:             443.99 GB
Size after:              277.36 GB
Space saved:             166.62 GB (37.5%)

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ PROCESSING COMPLETE                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Items: 405,317,030 (estimated)                                                                   │
│ Size: 277.36 GB                                                        

'/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db'

## **Step 3: Pivot to Yearly Indices**

In [4]:
build_pivoted_db(
    mode="restart",
    ngram_size=size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    num_workers=40,
    num_initial_work_units=600,
    cache_partitions=True,
    use_cached_partitions=True,
    progress_every_s=10.0,
);


PARALLEL N-GRAM DATABASE PIVOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-18 03:48:56
Mode:       RESTART

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            ...dk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Target DB:            .../edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db
Temp directory:       /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/pivoting_tmp

Parallelism
────────────────────────────────────────────────────────────────────────────────────────────────────
Workers:              40
Initial work units:   600

Database Profiles
────────────────────────────────────────────────────────────────────────────────────────────────────
Reader profile:       read:packed24
Writer profile:       write:packed24
Ingest profile:       write:packed24



SST Files Ingested: 100%|████████████████████████████████████████████████████| 600/600 [00:39<00:00]



Phase 4: Finalizing database...
════════════════════════════════════════════════════════════════════════════════════════════════════

Post-Ingestion Compaction
────────────────────────────────────────────────────────────────────────────────────────────────────
Initial DB size:         175.11 GB
Compaction completed in 0:33:37
Size before:             175.11 GB
Size after:              429.77 GB
Space saved:             -254.66 GB (-145.4%)

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ PROCESSING COMPLETE                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Items: 12,155,481,657 (estimated)                                                                │
│ Size: 429.77 GB                                                                                  │
│ Database: /scratch/edk202/NLP_corpora/Google_Bo

# Inspect Final Database
Here are three functions you can use to inspect the final database.

## `db_head`: First N records

In [5]:
pivoted_db = f'{db_path_stub}{release}/{language}/{size}gram_files/{size}grams_pivoted.db'

db_head(pivoted_db, n=5)

First 5 key-value pairs:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [1470] <UNK> <UNK> <UNK> <UNK> convenient
     Value: 1 occurrences in 1 documents

[ 2] Key:   [1470] <UNK> <UNK> <UNK> <UNK> eng
     Value: 1 occurrences in 1 documents

[ 3] Key:   [1470] <UNK> <UNK> <UNK> atomic energy
     Value: 1 occurrences in 1 documents

[ 4] Key:   [1470] <UNK> <UNK> <UNK> convenient form
     Value: 1 occurrences in 1 documents

[ 5] Key:   [1470] <UNK> <UNK> <UNK> convenient one
     Value: 1 occurrences in 1 documents



## `db_peek`: Records starting from a key

In [6]:
pivoted_db = f'{db_path_stub}{release}/{language}/{size}gram_files/{size}grams_pivoted.db'

db_peek(pivoted_db, start_key="[2002] attack <UNK> <UNK> world trade", n=5)

5 key-value pairs starting from 000007d261747461636b203c554e4b3e203c554e4b3e20776f726c64207472616465:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2002] attack <UNK> <UNK> world trade
     Value: 6,449 occurrences in 3,511 documents

[ 2] Key:   [2002] attack <UNK> <UNK> world war
     Value: 5 occurrences in 5 documents

[ 3] Key:   [2002] attack <UNK> <UNK> worm <UNK>
     Value: 1 occurrences in 1 documents

[ 4] Key:   [2002] attack <UNK> <UNK> worship <UNK>
     Value: 7 occurrences in 7 documents

[ 5] Key:   [2002] attack <UNK> <UNK> worst <UNK>
     Value: 8 occurrences in 8 documents



## `db_peek_prefix`: Records matching a prefix

In [7]:
pivoted_db = f'{db_path_stub}{release}/{language}/{size}gram_files/{size}grams_pivoted.db'

db_peek_prefix(pivoted_db, prefix="[2011] poor <UNK> happy", n=5)

5 key-value pairs with prefix 000007db706f6f72203c554e4b3e206861707079:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2011] poor <UNK> happy <UNK> <UNK>
     Value: 91 occurrences in 78 documents

[ 2] Key:   [2011] poor <UNK> happy <UNK> rich
     Value: 11 occurrences in 11 documents

[ 3] Key:   [2011] poor <UNK> happy <UNK> sad
     Value: 20 occurrences in 19 documents

[ 4] Key:   [2011] poor <UNK> happy <UNK> unhappy
     Value: 13 occurrences in 13 documents

[ 5] Key:   [2011] poor <UNK> happy childhood <UNK>
     Value: 7 occurrences in 7 documents

